In [11]:
# Import required Libraries
# !pip install fastparquet
import math
import pandas_datareader as web
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import datetime

In [12]:
stocks = pd.read_parquet('https://github.com/ddkpham/Stock_Prediction_NLP_ML/blob/master/wsb.hourly.joined.parquet.gz?raw=true')

In [13]:
stocks = stocks.set_index('created_utc')
stocks.head(5)

,score,gilded,wordcount,0,1,2,3,4,5,6,...,20,21,22,23,24,AAPL_StockPrice,AMZN_StockPrice,BA_StockPrice,SPY_StockPrice,TSLA_StockPrice
created_utc,,,,,,,,,,,,,,,,,,,,,
2017-01-03 00:00:00,10.600000,0.0,15.127273,-0.004773,0.096914,-0.028997,-0.013654,-0.040472,-0.028197,0.125876,...,0.034497,0.012236,0.022241,0.046725,-0.052190,116.01,757.76,155.68,224.86,214.3
2017-01-03 01:00:00,6.126761,0.0,14.957746,-0.018255,0.086793,-0.015693,-0.014463,-0.047337,-0.039189,0.151025,...,0.004107,0.018103,0.037762,0.030239,-0.058607,116.01,757.76,155.68,224.86,214.3
2017-01-03 02:00:00,5.150685,0.0,18.753425,-0.013361,0.085355,-0.045373,-0.029794,-0.013735,-0.030920,0.150325,...,-0.003617,0.002007,0.037377,0.017159,-0.064736,116.01,757.76,155.68,224.86,214.3
2017-01-03 03:00:00,5.723404,0.0,16.893617,-0.008933,0.079429,-0.009839,-0.026562,-0.041718,-0.015318,0.154539,...,-0.007581,0.005232,0.022578,0.016312,-0.054883,116.01,757.76,155.68,224.86,214.3
2017-01-03 04:00:00,1.516667,0.0,19.700000,-0.015290,0.079454,-0.002097,-0.019933,-0.023108,-0.015315,0.154939,...,-0.011684,0.011437,0.028106,0.017372,-0.061380,116.01,757.76,155.68,224.86,214.3


In [14]:
dataset = stocks.values
dataset

array([[1.06000000e+01, 0.00000000e+00, 1.51272727e+01, ...,
        1.55680000e+02, 2.24860000e+02, 2.14300000e+02],
       [6.12676056e+00, 0.00000000e+00, 1.49577465e+01, ...,
        1.55680000e+02, 2.24860000e+02, 2.14300000e+02],
       [5.15068493e+00, 0.00000000e+00, 1.87534247e+01, ...,
        1.55680000e+02, 2.24860000e+02, 2.14300000e+02],
       ...,
       [7.10147601e+00, 1.84501845e-03, 1.74298893e+01, ...,
        3.40090000e+02, 3.03330000e+02, 3.14600000e+02],
       [6.61122661e+00, 0.00000000e+00, 1.62993763e+01, ...,
        3.40090000e+02, 3.03330000e+02, 3.14600000e+02],
       [4.51340996e+00, 0.00000000e+00, 1.44923372e+01, ...,
        3.40090000e+02, 3.03330000e+02, 3.14600000e+02]])

In [15]:
price_data = stocks.filter(['AAPL_StockPrice', 'AMZN_StockPrice', 'BA_StockPrice', 'SPY_StockPrice', 'TSLA_StockPrice'])
price_dataset = price_data.values
price_dataset

array([[ 116.01,  757.76,  155.68,  224.86,  214.3 ],
       [ 116.01,  757.76,  155.68,  224.86,  214.3 ],
       [ 116.01,  757.76,  155.68,  224.86,  214.3 ],
       ...,
       [ 248.56, 1776.99,  340.09,  303.33,  314.6 ],
       [ 248.56, 1776.99,  340.09,  303.33,  314.6 ],
       [ 248.56, 1776.99,  340.09,  303.33,  314.6 ]])

In [16]:
# set training set len
training_data_len = math.ceil(len(dataset) * 0.8)

In [17]:
# Scale datasets 
scaler = MinMaxScaler(feature_range=(0,1))
price_scaler = MinMaxScaler(feature_range=(0,1))

scaled_dataset = scaler.fit_transform(dataset)
scaled_price_dataset = price_scaler.fit_transform(price_dataset)

In [18]:
# Split data into x_train, y_train 
train_features = np.array(scaled_dataset[0:training_data_len,:])
train_values = np.array(scaled_price_dataset[0:training_data_len, :])
x_train = []
y_train = []
for i in range(60, len(train_features)):
  x_train.append(train_features[i-60:i, :])
  y_train.append(train_values[i:i+1,:][0].tolist())
  if(i < 62):
    print(y_train)

[[0.010926312649164549, 0.021618826362464594, 0.010530677070745398, 0.02366032000455842, 0.21612732335951912]]
[[0.010926312649164549, 0.021618826362464594, 0.010530677070745398, 0.02366032000455842, 0.21612732335951912], [0.009396628878281632, 0.022344342134714368, 0.01102095777371126, 0.02477514582541085, 0.2183464545366125]]


In [19]:
# Convert both x, y training sets to np array 
x_train, y_train = np.array(x_train), np.array(y_train)
print(y_train, type(y_train))
# Reshape the data // LSTM network expects 3 dimensional input in the form of 
# (number of samples, number of timesteps, number of features)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 33))
x_train.shape

[[0.01092631 0.02161883 0.01053068 0.02366032 0.21612732]
 [0.00939663 0.02234434 0.01102096 0.02477515 0.21834645]
 [0.00809218 0.02339402 0.01112143 0.02477515 0.22986752]
 ...
 [0.6389469  0.85028133 0.72240445 0.79611074 0.4676446 ]
 [0.6389469  0.85028133 0.72240445 0.79611074 0.4676446 ]
 [0.6389469  0.85028133 0.72240445 0.79611074 0.4676446 ]] <class 'numpy.ndarray'>


(13558, 60, 33)

In [25]:
x_train.shape[1]

60

In [27]:
# Declare the LSTM model architecture 
model = Sequential()

# Add layers , input shape expected is (number of timesteps, number of features)
model.add(LSTM(75, return_sequences=True, input_shape=(x_train.shape[1], 33)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(5))

In [28]:
# Compile Model 
model.compile(optimizer='adam', loss='mean_squared_error')

In [29]:
# Train the model 
model.fit(x_train, y_train, batch_size=16, epochs=100)

Epoch 1/100


NotImplementedError: in user code:

    c:\users\kali\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    c:\users\kali\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\users\kali\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\users\kali\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\users\kali\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re

In [21]:
test_data = scaled_dataset[training_data_len-60:, :]
test_y = scaled_price_dataset[training_data_len-60:, :]

x_test = []
y_test = []

for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, :])
  y_test.append(test_y[i:i+1,:][0].tolist())

In [ ]:
# Convert the data into a np array
x_test = np.array(x_test)

# Reshape data into 3 dimensions ( num samples, timesteps, num features )
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 33))

In [ ]:
# Get the models predicted price values 
predictions = model.predict(x_test)

# Inverse transform 
predictions = price_scaler.inverse_transform(predictions)


In [ ]:
# Evaluate model with root mean square error (RMSE)
rmse = np.sqrt(np.mean((predictions-y_test)**2))
rmse

In [ ]:
# Set up ploting dataset
# plotData = stocks.filter(['created_utc','AAPL_CPrc', 'AMZN_CPrc', 'BA_CPrc', 'SPY_CPrc', 'TSLA_CPrc'])
# plotData = plotData.set_index("created_utc")
# print(plotData, type(plotData))

# This is so it graphs nicely with dates 
AAPL = stocks["AAPL_StockPrice"]
AAPL = AAPL.to_frame()
AMZN = stocks["AMZN_StockPrice"]
AMZN = AMZN.to_frame()
BA = stocks["BA_StockPrice"]
BA = BA.to_frame();
SPY = stocks["SPY_StockPrice"]
SPY = SPY.to_frame();
TSLA = stocks["TSLA_StockPrice"]
TSLA = TSLA.to_frame()

appleTest = AAPL[0:training_data_len]
amazonTest = AMZN[0:training_data_len]
boeTest = BA[0:training_data_len]
spyTest = SPY[0: training_data_len]
teslaTest = TSLA[0: training_data_len]

appleValid = AAPL[training_data_len:]
amazonValid = AMZN[training_data_len:]
boeValid = BA[training_data_len:]
spyValid = SPY[training_data_len:]
teslaValid = TSLA[training_data_len:]
# print(appleValid)

appleValid['Predictions'] = predictions[:,:1]
amazonValid["Predictions"] = predictions[:,1:2]
boeValid['Predictions'] = predictions[:, 2:3]
spyValid["Predictions"] = predictions[:, 3:4]
teslaValid["Predictions"] = predictions[:, 4:5]
print(len(appleValid), len(predictions)) # 3404 3404

In [ ]:
# Visualize data
plt.figure(figsize=(16,8))
plt.title('Prediction with reddit glove vectors')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
# plt.plot(appleTest)
# plt.plot(appleValid)
# plt.plot(appleValid['Predictions'])
# plt.plot(amazonTest)
# plt.plot(amazonValid)
# plt.plot(amazonValid['Predictions'])
# plt.plot(boeTest)
# plt.plot(boeValid)
# plt.plot(boeValid['Predictions'])
# plt.plot(spyTest)
# plt.plot(spyValid)
# plt.plot(spyValid['Predictions'])
plt.plot(teslaTest)
plt.plot(teslaValid)
plt.plot(teslaValid['Predictions'])
plt.legend(['Train', 'Validation', 'Prediction'])
plt.show()

In [ ]:
hourly_prices = stocks.filter(['AAPL_StockPrice', 'AMZN_StockPrice', 'BA_StockPrice', 
                               'SPY_StockPrice', 'TSLA_StockPrice'])
hourly_prices

In [ ]:
hourly_dataset = hourly_prices.values

In [ ]:
# set training set len
train_len = math.ceil(len(hourly_dataset) * 0.80) 

In [ ]:
hourScaler = MinMaxScaler(feature_range=(0,1))
scaled_hourly_prices = hourScaler.fit_transform(hourly_dataset)

In [ ]:
x_train_data = scaled_hourly_prices[0:train_len, :]

In [ ]:
x_train2 = []
y_train2 = []

for i in range(60, len(x_train_data)):
  x_train2.append(x_train_data[i-60:i, :])
  y_train2.append(x_train_data[i:i+1, :][0].tolist())


In [ ]:
# Convert both x,y training sets to np array
x_train2, y_train2 = np.array(x_train2), np.array(y_train2)
# Reshape the data // LSTM network expects 3 dimensional input in the form of 
# (number of samples, number of timesteps, number of features)
x_train2 = np.reshape(x_train2, (x_train2.shape[0], x_train2.shape[1], 5))
x_train2.shape

In [ ]:
# Declare the LSTM model architecture 
model2 = Sequential()

# Add layers , input shape expected is (number of timesteps, number of features)
model2.add(LSTM(75, return_sequences=True, input_shape=(x_train2.shape[1], 5)))
model2.add(LSTM(50, return_sequences=False))
model2.add(Dense(25))
model2.add(Dense(5))

In [ ]:
# Compile Model 
model2.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model 
model2.fit(x_train, y_train, batch_size=16, epochs=100)

In [ ]:
x_test_data = scaled_hourly_prices[train_len-60: , :]

In [ ]:
x_test2 = []
y_test2 = []
# y_test2 = scaled_hourly_prices[train_len:, :]
for i in range(60, len(x_test_data)):
  x_test2.append(x_test_data[i-60:i, :])
  y_test2.append(x_test_data[i:i+1,:][0].tolist())

print(len(y_test2))

In [ ]:
x_test2 = np.array(x_test2)

In [ ]:
x_test2 = np.reshape(x_test2, (x_test2.shape[0], x_test2.shape[1], 5))

In [ ]:
# Get the models predicted price values 
predictions2 = model2.predict(x_test2)

# Inverse transform 
predictions2 = hourScaler.inverse_transform(predictions2)
print(len(predictions2))

In [ ]:
# Evaluate model with root mean square error (RMSE)
rmse2 = np.sqrt(np.mean((predictions2-y_test2)**2))
rmse2

In [ ]:
print(len(appleValid)) # 3404
appleValid['Predictions2'] = predictions2[:,:1]
amazonValid["Predictions2"] = predictions2[:,1:2]
boeValid['Predictions2'] = predictions2[:, 2:3]
spyValid["Predictions2"] = predictions2[:, 3:4]
teslaValid["Predictions2"] = predictions2[:, 4:5]

In [ ]:
# Visualize data
plt.figure(figsize=(16,8))
plt.title('Stock Prediction with and without Reddit word vector data \n (AAPL)')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(appleTest)
plt.plot(appleValid)
plt.plot(appleValid['Predictions'], color='red')
plt.plot(appleValid['Predictions2'], color='green')
# plt.plot(amazonTest)
# plt.plot(amazonValid)
# plt.plot(amazonValid['Predictions'], color='red')
# plt.plot(amazonValid['Predictions2'], color='green')
# plt.plot(boeTest)
# plt.plot(boeValid)
# plt.plot(boeValid['Predictions'], color='red')
# plt.plot(boeValid['Predictions2'], color='green')
# plt.plot(spyTest, label='Train')
# plt.plot(spyValid, label='Actual')
# plt.plot(spyValid['Predictions'], label='Prediction with vector', color='red')
# plt.plot(spyValid['Predictions2'], label='Prediction without vector', color='green')

# plt.plot(teslaTest, label='Train')
# plt.plot(teslaValid, label="Actual")
# plt.plot(teslaValid['Predictions'], label="vector prediction", color='red')
# plt.plot(teslaValid['Predictions2'], label="without vector prediction", color="green")
plt.legend(('Train', 'Validation', 'Prediction with vector', 'Prediction without'), title="Legend",)
plt.show()
